In [ ]:
pip install splinter --user


In [ ]:
from splinter import Browser
from selenium import webdriver
import os
import pandas as pd
import requests 
import pymongo
import json
from bs4 import BeautifulSoup as soup
from flask import Flask, jsonify, request
from flask_pymongo import PyMongo

In [ ]:
executable_path = {'executable_path': '/Users/danlarson/Documents/git/Trilogy/In-Class/Homework/12-Web-Scraping-and-Document-Databases/web-scraping-challenge/app/chromedriver'}


In [ ]:
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")
browser = Browser('chrome', **executable_path, headless=False, options=options)

In [ ]:
# https://docs.mongodb.com/manual/reference/default-mongodb-port/
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)


# Define the 'nasa_db' database in Mongo
db = client.nasa_db

In [23]:
mars_news = 'https://mars.nasa.gov/news/'

featured_images = 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html'
facts_url = 'https://space-facts.com/mars/'
mars_hem = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'


news_title = []
news_p = []
featured_image_url = []
hemisphere_image_urls = []
nasa_page = []


In [89]:
browser.visit(mars_news)


In [90]:
news_title.append(browser.find_by_css('.content_title')[1].text)


In [91]:
news_p.append(browser.find_by_css('.article_teaser_body').text)

In [92]:
print(news_title)
print(news_p)

["Seeing NASA's Ingenuity Mars Helicopter Fly in 3D"]
['A new video of the helicopter’s third flight gives viewers the sensation of standing on the Red Planet and seeing the action firsthand.']


In [93]:
browser.visit(featured_images)

In [94]:
# browser.find_by_css('.headerimage fade-in').click()
# button = browser.find_by_text(' FULL IMAGE')

button = browser.find_by_tag('button')[1]
button.click()




In [95]:
html = browser.html
img_url = soup(html, 'html.parser')
img = img_url.find('img', class_='fancybox-image').get('src')


In [96]:
print(img)

image/featured/mars2.jpg


In [97]:
featured_image_url = f'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/{img}'
print(featured_image_url)

https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/image/featured/mars2.jpg


In [98]:
mars_data = pd.read_html('https://space-facts.com/mars/')

In [101]:
mars_df = pd.DataFrame(mars_data[1])
mars_df.set_index('Mars - Earth Comparison', inplace=True)
mars_df

KeyError: "None of ['Mars - Earth Comparison'] are in the columns"

In [75]:
mars_df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n      <th>Earth</th>\n    </tr>\n    <tr>\n      <th>Mars - Earth Comparison</th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Diameter:</th>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>Distance from Sun:</th>\n      <td>227,943,824 km</td>\n      <td>149,598,262 km</td>\n    </tr>\n    <tr>\n      <th>Length of Year:</th>\n      <td>687 Earth days</td>\n      <td>365.24 days</td>\n    </tr>\n    <tr>\n      <th>Temperature:</th>\n      <td>-87 to -5 °C</td>\n      <td>-88 to 58°C</td>\n    </tr>\n  </tbody>\n</table>'

In [ ]:
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'

browser.visit(url)

In [ ]:
hem_img = browser.find_by_css('a.product-item img')

In [34]:
for i in range(len(hem_img)):
    hem = {}
    browser.find_by_css('a.product-item img')[i].click()
    samp = browser.links.find_by_text('Sample').first
    hem['img_url'] = samp['href']
    hem['title'] = browser.find_by_css('h2.title').text
    hemisphere_image_urls.append(hem)
    browser.back()

In [ ]:
print(hemisphere_image_urls)

In [37]:
hemisphere_image_urls

[{'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
  'title': 'Cerberus Hemisphere Enhanced'},
 {'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
  'title': 'Schiaparelli Hemisphere Enhanced'},
 {'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
  'title': 'Syrtis Major Hemisphere Enhanced'}]

In [73]:
browser.quit()